In [1]:
from mmcv import Config
from mmdet.datasets import build_dataloader, build_dataset
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pycocotools.coco import COCO

In [2]:
model_cfg = \
"htc_x101_64x4d_fpn_dconv_c3-c5_mstrain_400_1400_16x1_20e_coco_all_data_instaboost"

# load config.
cfg = Config.fromfile(os.path.join("save_dir", model_cfg, "configs.py"))

In [3]:
# checkpoint
checkpoint_path = os.path.join(cfg.work_dir, "best_bbox_mAP_50.pth")

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=cfg.data.workers_per_gpu, dist=False, shuffle=False)

model = build_detector(cfg.model, test_cfg=cfg.get("test_cfg"))
checkpoint = load_checkpoint(model, checkpoint_path, map_location="cpu")

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Use load_from_local loader


In [4]:
output = single_gpu_test(model, data_loader, show_score_thr=0.001)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 837/837, 2.9 task/s, elapsed: 291s, ETA:     0s

In [5]:
try:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
except:
    prediction_strings = []
    file_names = []
    coco = COCO(cfg.data.test.ann_file)
    imag_ids = coco.getImgIds()

    class_num = 11
    for i, out in enumerate(output):
        prediction_string = ""
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for j in range(class_num):
            for o in out[0][j]:
                prediction_string += str(j) + " " + str(o[4]) + " " + str(o[0]) + " " + str(o[1]) + " " + str(o[2]) + " " + str(o[3]) + " "

        prediction_strings.append(prediction_string)
        file_names.append(image_info["file_name"])
    
submission = pd.DataFrame()
submission["PredictionString"] = prediction_strings
submission["image_id"] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f"submission_{model_cfg}.csv"), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


,PredictionString,image_id
0,1 0.03918961 166.52493 75.445 254.63074 164.61...,batch_01_vt/0021.jpg
1,0 0.0070311073 285.02002 342.14825 307.5397 37...,batch_01_vt/0028.jpg
2,0 0.0023195667 236.0474 461.0528 302.72284 510...,batch_01_vt/0031.jpg
3,0 0.0024504028 0.08807373 225.09482 33.78601 3...,batch_01_vt/0032.jpg
4,1 0.65325046 388.14993 275.3894 472.24924 344....,batch_01_vt/0070.jpg
